# Homework 2

In [53]:
import csv
import jellyfish
import pandas as pd
import re
import rltk
import uuid

from Levenshtein import distance
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF

### Task 1-1

In [2]:
df1 = pd.read_csv("/Users/kevin/Desktop/DSCI558/Homework/hw02/imdb_movies.csv")
df2 = pd.read_csv("/Users/kevin/Desktop/DSCI558/Homework/hw02/rotten_tomatoes_movies.csv")

tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

/var/folders/45/pv_8kn_n6k70y2wfr3kw43n40000gn/T/ipykernel_7403/801737449.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/Users/kevin/Desktop/DSCI558/Homework/hw02/imdb_movies.csv")


In [3]:
def important_words(title):
    title = title.lower()
    title = re.sub(r'[^a-z0-9\s]', '', title)
    if re.search(r'\d+', title):
        return re.search(r'\d+', title).group()
    else:
        words = title.split()
        words = sorted(words)
        return max(words,key=len) if words else ""

In [4]:
#df1.head()
# relevant columns in order 1)startYear 2)important word
# ID = tconst

In [5]:
#df2.head()
# relevant columns in order 1)year(original_release_date) 2)important word
# ID = rotten_tomatoes_link

In [6]:
class imdb_record(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''
    @rltk.cached_property
    def id(self):
        return self.raw_object['tconst']
    @rltk.cached_property
    def title_str(self):
        return self.raw_object['primaryTitle'].lower()
    @rltk.cached_property
    def yr(self):
        return self.raw_object['startYear'] 
    @rltk.cached_property
    def keyword(self):
        return important_words(self.raw_object['primaryTitle'])
    @rltk.cached_property
    def genre(self):
        return self.raw_object['genres'] 
    
class rt_record(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''
    @rltk.cached_property
    def id(self):
        return self.raw_object['rotten_tomatoes_link']
    @rltk.cached_property
    def title_str(self):
        return self.raw_object['movie_title'].lower()
    @rltk.cached_property
    def yr(self):
        return self.raw_object['original_release_date'].split('-')[0] 
    @rltk.cached_property
    def keyword(self):
        return important_words(self.raw_object['movie_title'])
    @rltk.cached_property
    def genre(self):
        return self.raw_object['genres'] 

In [7]:
dir_ = ''
imdb_file = dir_ + 'imdb_movies.csv'
rt_file = dir_ + 'rotten_tomatoes_movies.csv'

imdb = rltk.Dataset(rltk.CSVReader(imdb_file),record_class=imdb_record)
rt = rltk.Dataset(rltk.CSVReader(rt_file),record_class=rt_record)

In [8]:
#print(imdb.generate_dataframe().head(10))
#print(rt.generate_dataframe().head(10))

### Task 1-2

In [9]:
bg = rltk.HashBlockGenerator()
block_yr = bg.generate(bg.block(imdb, property_='yr'), 
                       bg.block(rt, property_='yr')
)

bg2 = rltk.HashBlockGenerator()
block_yr_iw = bg2.generate(bg2.block(imdb, property_='keyword', base_on = block_yr), 
                           bg2.block(rt, property_='keyword', base_on = block_yr)
)

In [10]:
pairs = rltk.get_record_pairs(imdb, rt, block=block_yr_iw)

r1_id = []
r2_id = []
r1_title = []
r2_title = []
r1_genres = []
r2_genres = []

for r1, r2 in pairs:
    r1_id.append(r1.id)
    r2_id.append(r2.id)
    r1_title.append(r1.title_str)
    r2_title.append(r2.title_str)
    r1_genres.append(r1.genre)
    r2_genres.append(r2.genre)

pairs_results = pd.DataFrame()
pairs_results['tconst'] = r1_id
pairs_results['rotten_tomatoes_link'] = r2_id

pairs_results.to_csv('Kevin_OKane_blocked.csv')

pairs_df = pd.DataFrame()
pairs_df['tconst'] = r1_id
pairs_df['rotten_tomatoes_link'] = r2_id
pairs_df['imdb_title'] = r1_title
pairs_df['rt_title'] = r2_title
pairs_df['imdb_genres'] = r1_genres
pairs_df['rt_genres'] = r2_genres

In [11]:
dev = pd.read_csv("dev_set.csv")
gt = dev[dev['label'] == 1]
comp1 = pairs_results[pairs_results['tconst'].isin(gt['tconst'])]
comp2 = pairs_results[pairs_results['rotten_tomatoes_link'].isin(gt['rotten_tomatoes_link'])]
comp = pd.concat([comp1,comp2])

gt_set = set(zip(gt['tconst'],gt['rotten_tomatoes_link']))
comp_set = set(zip(comp['tconst'],comp['rotten_tomatoes_link']))

reduction_ratio = 1-(len(pairs_results)/(len(df1)*len(df2)))
pairs_completeness = len(gt_set & comp_set)/len(gt_set)
pairs_quality = len(gt_set & comp_set)/len(comp_set)

print(f'Reduction Ratio: {round(reduction_ratio,3)}\nPair Completeness (Recall): {round(pairs_completeness,3)}\nPair Completeness (Precision): {round(pairs_quality,3)}')

Reduction Ratio: 1.0
Pair Completeness (Recall): 0.973
Pair Completeness (Precision): 0.507


### Task 1-3

In [12]:
comp_names = pd.concat([pairs_df[pairs_df['tconst'].isin(gt['tconst'])],pairs_df[pairs_df['rotten_tomatoes_link'].isin(gt['rotten_tomatoes_link'])]])
comp_names = comp_names.drop_duplicates()
comp_names = comp_names.reset_index()

In [13]:
comp_names.head()

,index,tconst,rotten_tomatoes_link,imdb_title,rt_title,imdb_genres,rt_genres
0,2540,tt0236388,m/original_kings_of_comedy,the original kings of comedy,the original kings of comedy,"Comedy,Documentary","Comedy, Documentary"
1,2575,tt0177971,m/perfect_storm,the perfect storm,the perfect storm,"Action,Adventure,Drama","Action & Adventure, Drama"
2,3060,tt0236216,m/filth_and_the_fury,the filth and the fury,the filth and the fury,"Biography,Documentary,Music","Documentary, Musical & Performing Arts"
3,3222,tt0107206,m/in_the_line_of_fire,in the line of fire,in the line of fire,"Action,Crime,Drama","Action & Adventure, Mystery & Suspense"
4,3223,tt0107206,m/fire_in_the_sky,in the line of fire,fire in the sky,"Action,Crime,Drama","Drama, Mystery & Suspense, Science Fiction & F..."


In [14]:
#Levenshtein Distance

count = 0
for i in range(len(comp_names)):
    lev_dist = distance(comp_names['imdb_title'][i], comp_names['rt_title'][i])
    if lev_dist == 0: count += 1
    elif lev_dist < min(len(comp_names['imdb_title'][i]), len(comp_names['rt_title'][i]))/3: 
        count += 1
        #print(comp_names['imdb_title'][i], comp_names['rt_title'][i], lev_dist)
    else: continue
#print(count)

#print(df1[df1['primaryTitle'].str.lower() == 'motherland'])
#print(df2[df2['movie_title'].str.lower() == 'motherland'])
#pretty good

In [15]:
#Jaccard Similarity 

count = 0
for i in range(len(comp_names)):
    imdb1 = set(comp_names['imdb_title'][i].split())
    rt1 = set(comp_names['rt_title'][i].split())
    
    intersection = len(imdb1.intersection(rt1))
    union = len(imdb1.intersection(rt1))

    if union == 0:
        continue
    elif intersection/union == 1:
       #print(imdb1, rt1)
        count += 1
    else:
        continue
        #print(imdb1, rt1, intersection/union)

#print(count)
#no good

In [16]:
# Jaro-Winkler

match_confidence = []
for i in range(len(comp_names)):
    jw_dist = jellyfish.jaro_winkler_similarity(comp_names['imdb_title'][i], comp_names['rt_title'][i])
    match_confidence.append(jw_dist)
    if jw_dist ==1:
        continue #print(comp_names['imdb_title'][i], '|' ,comp_names['rt_title'][i], '|' ,jw_dist)
#print(count)
#pretty good

In [17]:
comp_names['match_confidence'] = match_confidence
comp_names.head()

,index,tconst,rotten_tomatoes_link,imdb_title,rt_title,imdb_genres,rt_genres,match_confidence
0,2540,tt0236388,m/original_kings_of_comedy,the original kings of comedy,the original kings of comedy,"Comedy,Documentary","Comedy, Documentary",1.000000
1,2575,tt0177971,m/perfect_storm,the perfect storm,the perfect storm,"Action,Adventure,Drama","Action & Adventure, Drama",1.000000
2,3060,tt0236216,m/filth_and_the_fury,the filth and the fury,the filth and the fury,"Biography,Documentary,Music","Documentary, Musical & Performing Arts",1.000000
3,3222,tt0107206,m/in_the_line_of_fire,in the line of fire,in the line of fire,"Action,Crime,Drama","Action & Adventure, Mystery & Suspense",1.000000
4,3223,tt0107206,m/fire_in_the_sky,in the line of fire,fire in the sky,"Action,Crime,Drama","Drama, Mystery & Suspense, Science Fiction & F...",0.630994


In [18]:
dev_csv_df = dev

titles1 = []
titles2 = []

for i in range(len(dev_csv_df)):
    titles1.append(df1[df1['tconst'] == dev_csv_df['tconst'][i]]['primaryTitle'].values[0])
    titles2.append(df2[df2['rotten_tomatoes_link'] == dev_csv_df['rotten_tomatoes_link'][i]]['movie_title'].values[0])

dev_csv_df['imdb_title'] = titles1
dev_csv_df['rt_title'] = titles2

pred = []
conf = []

for i in range(len(dev_csv_df)):
    jw_dist = jellyfish.jaro_winkler_similarity(dev_csv_df['imdb_title'][i].lower(), dev_csv_df['rt_title'][i].lower())
    if jw_dist == 1:
        pred.append(1)
        conf.append(1)
    elif jw_dist >= 0.95:
        pred.append(1)
        conf.append(jw_dist)
    elif jw_dist <= 0.7:
        pred.append(0)
        conf.append(1)
    else:
        pred.append(0)
        conf.append(jw_dist + (0.95-jw_dist))

dev_csv_df['prediction'] = pred
dev_csv_df['confidence'] = conf

Kevin_OKane_predictions_df = dev_csv_df[['tconst','rotten_tomatoes_link','prediction','confidence']]
Kevin_OKane_predictions_df.to_csv('Kevin_OKane_predictions.csv',)

### Task 1-4

In [19]:
## dataframe of blocked pairs from Task 1-2
pairs_df.head()

,tconst,rotten_tomatoes_link,imdb_title,rt_title,imdb_genres,rt_genres
0,tt0006031,m/birth_of_a_nation,the shame of a nation,the birth of a nation,Drama,"Classics, Drama"
1,tt0004972,m/birth_of_a_nation,the birth of a nation,the birth of a nation,"Drama,War","Classics, Drama"
2,tt0005339,m/fool_there_was,a fool there was,a fool there was,Drama,"Classics, Drama"
3,tt0003740,m/cabiria,cabiria,cabiria,"Adventure,Drama,History","Art House & International, Classics, Drama"
4,tt0136580,m/spiders_part_1_the_the_golden_lake,der tänzer 1. teil,"spiders, part 1: the golden lake (die spinnen,...",Drama,"Action & Adventure, Western"


In [20]:
test_df = pd.read_csv('test_set.csv')
test_df.head()

,tconst,rotten_tomatoes_link
0,tt0091630,m/night_of_the_creeps
1,tt0110428,m/madness_of_king_george
2,tt0076929,m/wizards
3,tt0029982,m/mother-kusters-goes-to-heaven
4,tt0164052,m/hollow_man


In [21]:
titles1 = []
titles2 = []
genres1 = []
genres2 = []

for i in range(len(test_df)):
    try:
        titles1.append(pairs_df[pairs_df['tconst'] == test_df['tconst'][i]]['imdb_title'].values[0])
    except:
        titles1.append(pd.NA)
    try:
        titles2.append(pairs_df[pairs_df['rotten_tomatoes_link'] == test_df['rotten_tomatoes_link'][i]]['rt_title'].values[0])
    except:
        titles2.append(pd.NA)
    try:
        genres1.append(pairs_df[pairs_df['tconst'] == test_df['tconst'][i]]['imdb_genres'].values[0])
    except:     
        genres1.append(pd.NA)
    try: 
        genres2.append(pairs_df[pairs_df['rotten_tomatoes_link'] == test_df['rotten_tomatoes_link'][i]]['rt_genres'].values[0])
    except:
        genres2.append(pd.NA)

test_df['imdb_title'] = titles1
test_df['rt_title'] = titles2
test_df['imdb_genres'] = genres1
test_df['rt_genres'] = genres2

test_df.head()

,tconst,rotten_tomatoes_link,imdb_title,rt_title,imdb_genres,rt_genres
0,tt0091630,m/night_of_the_creeps,night of the creeps,night of the creeps,"Comedy,Horror,Sci-Fi","Comedy, Horror, Science Fiction & Fantasy"
1,tt0110428,m/madness_of_king_george,the madness of king george,the madness of king george,"Biography,Comedy,Drama","Comedy, Drama"
2,tt0076929,m/wizards,wizards,wizards,"Adventure,Animation,Fantasy","Action & Adventure, Animation, Kids & Family, ..."
3,tt0029982,m/mother-kusters-goes-to-heaven,<NA>,mother kusters goes to heaven,<NA>,"Art House & International, Drama"
4,tt0164052,m/hollow_man,hollow man,hollow man,"Action,Horror,Sci-Fi","Action & Adventure, Horror, Science Fiction & ..."


In [22]:
jw_sim = []

for i in range(len(test_df)):
    if test_df['imdb_title'][i] is pd.NA or test_df['rt_title'][i] is pd.NA:
        jw_sim.append(0)
    else:
        jw_dist = jellyfish.jaro_winkler_similarity(test_df['imdb_title'][i].lower(), test_df['rt_title'][i].lower())
        jw_sim.append(jw_dist)

test_df['jw_score'] = jw_sim
matches = test_df[test_df['jw_score'] > 0.95]

In [23]:
matches = matches[['tconst','rotten_tomatoes_link']]
matches.to_csv("Kevin_OKane_valid_predictions.csv")

### Task 2-1

In [79]:
imdb_match = df1[df1['tconst'].isin(matches['tconst'])]
rt_match = df2[df2['rotten_tomatoes_link'].isin(matches['rotten_tomatoes_link'])]

imdb_match = imdb_match[['primaryTitle','startYear','genres','tconst','runtimeMinutes']]
rt_match = rt_match[['movie_title','original_release_date','genres','rotten_tomatoes_link','directors','tomatometer_rating','audience_rating','production_company']]

imdb_merged = pd.merge(imdb_match, matches, on='tconst', how='left')
KGdata = pd.merge(imdb_merged, rt_match, on= 'rotten_tomatoes_link', how='left')

KGdata.rename(columns={'genres_x':'genres_imdb', 'genres_y':'genres_rt'}, inplace=True)


In [80]:
NS1 = Namespace('http://schema.org')
MYNS = Namespace('http://dsci558.org/kevinsfakens')
kg = Graph()
kg.bind('schema',NS1)
kg.bind('myns',MYNS)

In [81]:
for i in range(len(KGdata)):
    movie_uri = URIRef(KGdata['tconst'][i])
    kg.add((movie_uri, RDF.type, NS1['movie']))
    kg.add((movie_uri, NS1['name'], Literal(KGdata['primaryTitle'][i])))
    kg.add((movie_uri, NS1['duration'], Literal(int(KGdata['runtimeMinutes'][i]))))
    kg.add((movie_uri, MYNS['tomatometer_rating'], Literal(float(KGdata['tomatometer_rating'][i]))))
    kg.add((movie_uri, MYNS['audience_rating'], Literal(float(KGdata['audience_rating'][i]))))
    kg.add((movie_uri, NS1['datePublished'], Literal(KGdata['original_release_date'][i])))
    kg.add((movie_uri, MYNS['year'], Literal(int(KGdata['startYear'][i]))))
    kg.add((movie_uri, MYNS['imdb_genres'], Literal(KGdata['genres_imdb'][i])))
    kg.add((movie_uri, MYNS['rotten_tomatoes_genres'], Literal(KGdata['genres_rt'][i])))
    kg.add((movie_uri, MYNS['rotten_tomatoes_link'], Literal(KGdata['rotten_tomatoes_link'][i])))
    director_uri = URIRef('d'+str([i]))
    kg.add((director_uri, RDF.type, NS1['director']))
    kg.add((director_uri, NS1['director'], Literal(KGdata['directors'][i])))
    kg.add((movie_uri, NS1['director'], director_uri))
    prod_co_uri = URIRef('pc'+str([i]))
    kg.add((prod_co_uri, RDF.type, NS1['productionCompany']))
    kg.add((prod_co_uri, NS1['productionCompany'], Literal(KGdata['production_company'][i])))
    kg.add((movie_uri, NS1['productionCompany'], prod_co_uri))

In [82]:
kg.serialize(dir_ + 'Kevin_OKane_model.ttl', format="turtle")

<Graph identifier=Ne0397bd321b343ccbf0f370b929c4ba9 (<class 'rdflib.graph.Graph'>)>